1,输入子系统的作用和框架

> 什么是输入设备：

1. 按键/keyboard
2. mouse
3. touchscreen ：gt811, ft56xx
4. joystick 游戏杆

> 有多个输入设备需要驱动的时候，假如不考虑输入子系统

1. gt811
		设备号，创建文件，硬件初始化，实现fop，阻塞
		硬件初始化
2. ft56xx
		设备号，创建文件，硬件初始化，实现fop，阻塞
		硬件初始化	

> 多个输入设备有共同点：
	
| | |
| -------------------- | --------------------- |
| 获取到数据(操作硬件) | 上报给用户（xxx_read, copy_to_user, 阻塞）|
|	   差异化        |               通用        |

多个输入设备，有部分差异，也有部分通用
内核就会考虑，将通用代码编写好，将差异化的代码留给驱动工程师

> 设计成输入子系统：使得应用编程人员和驱动编程人员编程的时候变得简单统一  

1. 兼容所有的输入设备  
2. 统一的编程驱动方法(实现差异化硬件操作)  
3. 统一的应用操作接口：  
/dev/input/event0,event1   
open("/dev/input/event0")  
read(fd, struct input_event): struct input_event buff可以认为是一个统一的数据包输入子系统的作用  


>框架：驱动分成三层

```c
            应用层
    ---------------------------------
    input handler层：数据处理者

        完成fop：实现xxx_read(), xxx_open
        将数据交给用户：数据从input device层
        不知道具体数据是什么，只知道把数据给用户
    ----------------------------------------------------------
    input　核心层：管理层
    ----------------------------------------------------------
    input　device设备层:
    
        抽象出一个对象，描述输入设备信息
        初始化输入设备硬件，获取到数据
        知道具体的数据是什么，但是不知道数据如何给用户
    ---------------------------------
    硬件层：mouse
            ts， keybaord，joystick

    编程： 主要在input device层
```

input子系统编程的前提：
input 核心层代码和input handler层需要在内核中必须有：  
		drivers/input/evdev.c	//  event handler  
		drivers/input/input.c  // 核心层  

```c
make menuconfig
Device Drivers  --->
	Input device support  ---> 
		-*- Generic input layer (needed for keyboard, mouse, ...)  // input.c
			<*>   Event interface   //input handler层--evdev.c
```

# 编写步骤：

## 通用的驱动代码

In [ ]:
static int __init simple_input_init(void)
{

	//编写输入子系统代码

	return 0;
}



static void __exit simple_input_exit(void)
{
    //注销申请的资源
}


## 分配一个input device对象

In [ ]:
struct input_dev *inputdev;

inputdev  =  input_allocate_device();
if(inputdev == NULL)
{
    printk(KERN_ERR "input_allocate_device error\n");
    return -ENOMEM;
}

## 初始化input device 对象

In [ ]:
//添加设备信息--/sys/class/input/eventx/device/
inputdev->name = "simple input key";
inputdev->phys = "key/input/input0";
inputdev->uniq = "simple key0 for 4412";
inputdev->id.bustype = BUS_HOST;
inputdev->id.vendor =0x1234 ;
inputdev->id.product = 0x8888;
inputdev->id.version = 0x0001;

> struct input_dev 结构体类

In [ ]:
struct input_dev //表示的是一个具体的输入设备，描述设备能够产生什么数据
{
	// sysfs中给用户看的信息
    const char *name; 
	const char *phys;
	const char *uniq;
    struct input_id id;
/*
 * @name: name of the device
 * @phys: physical path to the device in the system hierarchy
 * @uniq: unique identification code for the device (if device has it)
 * @id: id of the device (struct input_id)
 */
	//evbit实际是一个位表，描述输入设备能够产生什么类型数据
	unsigned long evbit[BITS_TO_LONGS(EV_CNT)]; // EV_KEY，EV_ABS, EV_REL
	//表示能够产生哪种按键
	unsigned long keybit[BITS_TO_LONGS(KEY_CNT)];//KEY_POWER.. 能够表示768bit，直接用24个long来表示
							                    // KEY_CNT == 768   BITS_TO_LONGS== nr/32 = 768/32==24
	//表示能够产生哪种相对坐标数据
	unsigned long relbit[BITS_TO_LONGS(REL_CNT)];// REL_X
	//表示能够产生哪种绝对坐标数据
	unsigned long absbit[BITS_TO_LONGS(ABS_CNT)]; //ABS_X
	unsigned long mscbit[BITS_TO_LONGS(MSC_CNT)];
	unsigned long ledbit[BITS_TO_LONGS(LED_CNT)];
	unsigned long sndbit[BITS_TO_LONGS(SND_CNT)];
	unsigned long ffbit[BITS_TO_LONGS(FF_CNT)];
	unsigned long swbit[BITS_TO_LONGS(SW_CNT)];

	struct device dev; // 继承device对象

	struct list_head	h_list;
	struct list_head	node; //表示节点
}

In [ ]:
> 不同输入设备能够产生不同的数据：

1. 按键/keyboard： 产生键值，实际是一个数字
        #define KEY_VOLUMEDOWN		114
        #define KEY_VOLUMEUP		115
        #define KEY_POWER		116	/* SC System Power Down */
2. ts/gsensor:产生坐标，绝对坐标， 有一个明确坐标系，并且原点(0,0),最大值(800,480)
        #define ABS_X			0x00
        #define ABS_Y			0x01
        #define ABS_PRESSURE		0x18
        #define ABS_MT_TOUCH_MAJOR	0x30	/* Major axis of touching ellipse */
        #define ABS_MT_TOUCH_MINOR	0x31	/* Minor axis (omit if circular) */
        #define ABS_MT_WIDTH_MAJOR	0x32	/* Major axis of approaching ellipse */
        #define ABS_MT_WIDTH_MINOR	0x33	/* Minor axis (omit if circular) */
        #define ABS_MT_ORIENTATION	0x34	/* Ellipse orientation */
        #define ABS_MT_POSITION_X	0x35	/* Center X touch position */
        #define ABS_MT_POSITION_Y	0x36	/* Center Y touch position */
3. mouse：产生坐标，相对坐标，坐标值是相对于之前一个点坐标	
        #define REL_X			0x00
        #define REL_Y			0x01
        #define REL_WHEEL		0x08

> 如何表示不同数据类型： 

    #define EV_SYN			0x00 //表示同步数据类型
    #define EV_KEY			0x01 //表示按键数据类型
    #define EV_REL			0x02 //表示相对坐标数据类型
    #define EV_ABS			0x03 //表示绝对坐标数据类型

    #define EV_MSC			0x04 //表示杂项
    #define EV_SW			0x05 //开关  
    #define EV_LED			0x11 //led指示数据
    #define EV_SND			0x12 //声音数据


设置bit的方法

In [ ]:
//当前设备能够产生按键数据
__set_bit(EV_KEY,  inputdev->evbit);
//表示当前设备能够产生power按键
__set_bit(KEY_POWER, inputdev->keybAit);

In [ ]:
inputdev->keybit[BIT_WORD(KEY_POWER)] |= BIT_MASK(KEY_POWER);// 116%32
inputdev->keybit[116/32] |= 1 << 116%32;// 116%32